In [1]:
# app/visualizations.py
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
file_path = '/workspaces/Feedback_Error_Chatbot/data/predicting_students_errors.csv'

In [3]:
df = pd.read_csv(file_path)

In [4]:
def grade_distribution(df):
    df['grade'] = df['grade'].astype(float)
    plt.figure(figsize=(8, 5))
    sns.histplot(df['grade'], bins=10, kde=True)
    plt.title("Grade Distribution")
    plt.xlabel("Grade")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [5]:
grade_distribution(df)

In [6]:
from dotenv import load_dotenv
import os


In [7]:
%load_ext dotenv
%dotenv

In [8]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
load_dotenv()


True

In [9]:
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [10]:
NEO4J_URI

'neo4j+s://e1356d44.databases.neo4j.io'

In [11]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://e1356d44.databases.neo4j.io"
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [12]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri=URI, auth=AUTH)

try:
    with driver.session() as session:
        result = session.run("RETURN 'Neo4j connection successful' AS message")
        print(result.single()["message"])
except Exception as e:
    print("❌ Connection failed:", e)

driver.close()

Neo4j connection successful


In [15]:
import pandas as pd

df = pd.read_csv("/workspaces/Feedback_Error_Chatbot/data/predicting_students_errors.csv")
df = df.dropna(subset=["error_summary", "student_id", "question"])


In [16]:
df

,student_id,response,right_answer,question,seed_id,grade,llm_response,error_summary,error_category
0,1,"ans1: x^2+y^2=4 ans2: ""Point"" ans3: ""Circle""","ans1: y^2+x^2 = 4 ans2: ""Point"" ans3: ""Circl...",Q. 1,854304650,0.60,Based on the provided response and the right a...,omission error,"comprehension errors, encoding errors"
1,1,"ans1: y=18*t^2+9*t-1 ans2: ""Point"" ans3: ""Pa...","ans1: y = 2*x^2+3*x-1 ans2: ""Point"" ans3: ""P...",Q. 2,1225488973,0.40,To categorize the types of errors made by the ...,"misinterpretation of notation, incorrect numer...","comprehension errors, transformation errors, e..."
2,1,ans1: t/3 ans2: 3*t^2 ans3: 0 ans4: 5*sin(t...,ans1: t/3 ans2: 3*t^2 ans3: 0 ans4: 5*sin(t...,Q. 3,1509813580,0.75,Based on the comparison between the student's ...,incorrect numerical value,transformation errors
3,1,ans1: y=-8/5-8/5,ans1: y = -((8*x)/5)-8/5,Q. 4,163677917,0.00,"To analyze the student's error in mathematics,...","omission error, conceptual misunderstanding","comprehension errors, transformation errors"
4,1,"ans1: 2/(4*t^2-3*t-4) ans2: [t=2,t=-1,t=0] a...","ans1: (2*t)/(4*t^3-3*t^2-4*t) ans2: [t = 2,t ...",Q. 5,1500539490,0.57,The student's errors can be categorized as fol...,misinterpretation or incorrect transcription o...,"comprehension errors, transformation errors, e..."
...,...,...,...,...,...,...,...,...,...
2826,618,"ans1: x^2+y^2=1 ans2: ""Point"" ans3: ""Circle""...","ans1: y^2+x^2 = 1 ans2: ""Point"" ans3: ""Circl...",Q. 1,1504181159,0.80,Based on the comparison between the student's ...,omission error,encoding errors
2827,618,"ans1: y=-4*x^2-5*x ans2: ""Point"" ans3: ""Para...","ans1: y = -x^2-5*x+4 ans2: ""Point"" ans3: ""Pa...",Q. 2,141996950,0.40,To categorize the types of errors made by the ...,"incorrect numerical coefficient, omission erro...","transformation errors, process skill errors, c..."
2828,618,"ans1: t ans2: 3*t^2 ans3: ""cos^3*(theta)"" a...",ans1: t ans2: 3*t^2 ans3: cos(theta)^3 ans4: 0,Q. 3,896417394,0.75,Based on the information provided for the comp...,misinterpretation of notation,encoding errors
2829,618,"ans1: (2*t)/(5*t^4-8*t^3-9*t^3) ans2: [t=0,t=...","ans1: (2*t)/(5*t^4-8*t^3-9*t^2) ans2: [t = 3,...",Q. 5,353452773,0.14,Let's analyze the student's responses compared...,"misinterpretation of notation, incorrect numer...","comprehension errors, transformation errors, p..."


In [17]:
texts = df[["error_summary", "llm_response"]].fillna("").agg(" ".join, axis=1).tolist()
metadatas = df[["student_id", "question", "grade", "error_category"]].to_dict("records")


In [18]:
ids = [f"row-{i}" for i in range(len(df))]


In [19]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

chroma_client = chromadb.Client(chromadb.config.Settings(persist_directory="./chroma"))

collection = chroma_client.get_or_create_collection(
    name="student_errors",
    embedding_function=embedding_fn
)

collection.add(documents=texts, metadatas=metadatas, ids=ids)


/workspaces/Feedback_Error_Chatbot/myvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
results = collection.query(
    query_texts=["why do students fail question 2?"],
    n_results=5
)

for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print("→", doc)
    print("Meta:", meta)


→ omission error By comparing the student's response and the right answer, we can categorize the types of errors made:

1. Omission Error:
   - The student omitted 'ansb: 0' in their response. The right answer includes 'ansb: 0', which likely indicates sketching requirements that were not addressed by the student. 

2. Correct Numerical Value:
   - The student correctly identified 'ansa: 3', which matches the right answer. This indicates that the student understood the requirement for the radius of the circle.

3. Correct Conceptual Understanding:
   - The student correctly identified 'ans2: "Point"' and 'ans3: "Circle"', which match the right answers. This demonstrates that the student had the right conceptual understanding of the problem context.

Overall, the main error was an omission error, where the student left out a required part of the answer ('ansb: 0'). There are no misinterpretations of notation or incorrect numerical values in the provided student response.
Meta: {'questio

In [ ]:
streamlit run run_app.py
